In [1]:
# Instalar sentence-transformers y chromadb
!pip install sentence-transformers chromadb


In [2]:
import pandas as pd

# Cargar el archivo CSV con delimitador ;
file_path = 'Base_conocimiento_pre.csv'  # Actualiza el path si es necesario
df = pd.read_csv(file_path, delimiter=';')

# Mostrar las primeras filas del DataFrame
df.head()


,n,Pregunta,Respuesta
0,1,¿Dónde se encuentra el departamento de computa...,En el bloque H
1,2,¿Dónde se encuentra el psicólogo de la univers...,"En bienestar estudiantil, junto al bar"
2,3,¿Cada cuánto tiempo se da las becas economicas?,Una vez al año
3,4,¿Cuántos accesos exiten en la matriz de la uni...,Dos accesos para vehiculos y peatones
4,5,¿Que club existen en la universidad?,ASOCIACIÓN DE INGENIEROS MECÁNICOS “ASME”\nCHA...


In [3]:
# Limpiar los datos: eliminar filas con valores nulos en la columna 'Pregunta'
df = df.dropna(subset=['Pregunta'])

# Asegurarse de que todas las preguntas sean strings
df['Pregunta'] = df['Pregunta'].astype(str)

In [4]:
from sentence_transformers import SentenceTransformer

# Cargar el modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Crear embeddings para las preguntas
df['question_embeddings'] = df['Pregunta'].apply(lambda x: model.encode(x).tolist())


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
import chromadb
from chromadb.config import Settings
from chromadb import Client

# Configurar Chroma
settings = Settings()
client = Client(settings=settings)

# Crear una colección
collection = client.create_collection(name="qa_collection")

# Agregar vectores a la colección
for index, row in df.iterrows():
    collection.add(
        documents=[row['Pregunta']],
        embeddings=[row['question_embeddings']],
        ids=[str(index)] # Changed metadatas to ids
    )

In [6]:
def obtener_respuesta(pregunta: str, collection, df, model):
    """
    Obtiene la mejor respuesta para una pregunta dada usando la colección y el modelo de embeddings.

    Args:
    - pregunta (str): La pregunta para la cual obtener una respuesta.
    - collection: La colección de documentos en Chroma.
    - df: El DataFrame que contiene las preguntas y respuestas.
    - model: El modelo de embeddings para generar la representación de la pregunta.

    Returns:
    - str: La mejor respuesta correspondiente a la pregunta.
    """
    # Codificar la pregunta
    query_embedding = model.encode(pregunta).tolist()

    # Realizar la consulta en la colección
    results = collection.query(query_embedding)

    # Obtener las listas de IDs, distancias y documentos
    ids_list = results['ids'][0]
    distances_list = results['distances'][0]

    # Combina los IDs y distancias en un solo objeto
    results_combined = list(zip(ids_list, distances_list))

    # Ordena los resultados por distancia (menor distancia significa mayor similitud)
    sorted_results = sorted(results_combined, key=lambda x: x[1])

    # Selecciona solo el mejor resultado (menor distancia)
    best_result = sorted_results[0]  # Solo el primer resultado es el mejor

    doc_id, distance = best_result
    index = int(doc_id)

    # Obtener la pregunta y respuesta desde el DataFrame
    respuesta = df.loc[index, 'Respuesta']

    return respuesta


In [8]:
# Ejemplo de uso:
pregunta = "¿Cómo se llama el director de carrera de Software?"
respuesta = obtener_respuesta(pregunta, collection, df, model)
print(f"Respuesta: {respuesta}")

Respuesta: Ing. Mauricio Camapaña Alias "Monster"
